How to run Mono with docker?

In [1]:
FROM mono:latest



SyntaxError: Unexpected identifier

How to build a project with mono

In [2]:

echo '<configuration> \
    <packageRestore> \
        <add key="enabled" value="True" /> \
        <add key="automatic" value="True" /> \
    </packageRestore> \
    <packageSources> \
        <add key="nuget.org" value="https://www.nuget.org/api/v2/" /> \
        <add key="nuget.org" value="https://api.nuget.org/v3/index.json" protocolVersion="3" /> \
        <add key="teamCity" value="http://teamcity.actops.com/httpAuth/app/nuget/v1/FeedService.svc" /> \
    </packageSources> \
</configuration>' > /act.subscription.management/.nuget/NuGet.Config

ln -s subscription.myact/ Subscription.MyAct
ln -s subscription.core/ Subscription.Core
ln -s subscription.entitlement/ Subscription.Entitlement
ln -s subscription.integrations/ Subscription.Integrations
ln -s subscription.identitymanagement/ Subscription.IdentityManagement

nuget sources update -Name teamCity -source http://teamcity.actops.com/httpAuth/app/nuget/v1/FeedService.svc -User {username} -pass {password} -configFile .nuget/NuGet.Config -StorePasswordInClearText
nuget update /act.subscription.management/act.subscription.management.sln
msbuild Subscription.IdentityManagement.csproj


SyntaxError: Unexpected string

In [3]:
#FROM mono:latest

#RUN mono --version
#RUN ls -la
#ADD ./subscription.services /home/src
#ADD ./act.subscription.management.sln /home/src
#WORKDIR /home/src
#RUN msbuild ./subscription.services/Subscription.Services.EloquaImport/Subscription.Services.EloquaImport.csproj
#RUN mono Subscription.Services.EloquaImport.exe

#FROM microsoft/aspnetcore:1.1
FROM microsoft/aspnetcore-build:1.0-1.1
ARG source
EXPOSE 80
WORKDIR /app/src
ADD ./*.sln /app/src/
ADD ./subscription.services/Subscription.Services.EloquaImport/*.csproj /app/src/subscription.services/Subscription.Services.EloquaImport/
ADD ./subscription.services/Subscription.Services.ZuoraExport/*.csproj /app/src/subscription.services/Subscription.Services.ZuoraExport/
ADD ./*.dcproj /app/src/

RUN /bin/bash -c "dotnet restore ./act.subscription.management.sln"

ADD . /app/src 
RUN /bin/bash -c "dotnet restore ./act.subscription.management.sln && dotnet publish ./act.subscription.management.sln -c Release -o ./obj/Docker/publish"

RUN cp -R ${source:-subscription.services/Subscription.Services.EloquaImport/obj/Docker/publish} /app

WORKDIR /app/publish
EXPOSE 80
ENTRYPOINT ["/usr/bin/dotnet", "/app/publish/Subscription.Services.EloquaImport.dll"]



SyntaxError: Unexpected identifier